## Performance Test for Single-Model Inference
This notebook contains performance measurements for all Orca-based image detection AI models from DeGirum 
public model zoo

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

#### Specify test options here

In [ ]:
iterations = 100  # how many iterations to run for each model
device_family = "ORCA1"  # models of which device family to use

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, dgtools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = dgtools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, dgtools.get_cloud_zoo_url(), dgtools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
# list of models to test
model_names = zoo.list_models(device=device_family)

# run batch predict for each model and record time measurements
results = {}
prog = dgtools.Progress(len(model_names), speed_units="models/s")
for model_name in model_names:
    try:
        results[model_name] = dgtools.model_time_profile(
            zoo.load_model(model_name), iterations
        )
    except NotImplementedError:
        pass  # skip models for which time profiling is not supported
    prog.step()

In [ ]:
# print results
CW = (62, 19, 16, 16) # column widths
header = f"{'Model name':{CW[0]}}| {'Postprocess Type':{CW[1]}} | {'Observed FPS':{CW[2]}} | {'Max Possible FPS':{CW[3]}} |"

print(f"Models    : {len(model_names)}")
print(f"Iterations: {iterations}\n")
print(f"{'-'*len(header)}")
print(header)
print(f"{'-'*len(header)}")

for model_name, result in results.items():
    print(
        f"{model_name:{CW[0]}}|"
        + f" {result.parameters.OutputPostprocessType:{CW[1]}} |"
        + f" {result.observed_fps:{CW[2]}.1f} |"
        + f" {result.max_possible_fps:{CW[3]}.1f} |"
    )
